In [1]:
import pandas as pd

In [2]:
df1 = pd.read_stata("1314_sec_2ab.dta", convert_categoricals=False)

In [3]:
df1.sample(10)

,hhcode,psu,province,region,idc,s2aq01,s2aq02,s2aq03,s2bq01,s2bq02,...,s2bq19d,s2bq19e,s2bq19f,s2bq19g,s2bq19h,s2bq19i,filter__,hhcode_new,stratum,psu_new
46352,2.524100e+09,25241002,2,1,3,1.0,1.0,1.0,3,NaN,...,1800.0,4500.0,1200.0,0.0,0.0,12600.0,0,2.521040e+09,25210,25210402
12604,1.412200e+09,14122002,1,2,1,2.0,NaN,1.0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1.402020e+09,14020,14020202
72457,3.232100e+09,32321001,3,1,51,2.0,NaN,1.0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,3.231020e+09,32310,32310201
21363,1.712200e+09,17122001,1,2,52,1.0,1.0,1.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1.702020e+09,17020,17020201
101644,4.313100e+09,43131001,4,1,4,2.0,NaN,1.0,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,4.301030e+09,43010,43010301
30193,2.311220e+09,23112203,2,2,52,2.0,NaN,1.0,1,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,2.312210e+09,23122,23122103
8049,1.243100e+09,12431001,1,1,52,NaN,NaN,NaN,3,NaN,...,0.0,0.0,0.0,0.0,750.0,1220.0,1,1.241030e+09,12410,12410301
37168,2.414220e+09,24142203,2,2,51,1.0,1.0,1.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,2.412240e+09,24122,24122403
34049,2.334100e+09,23341003,2,1,54,NaN,NaN,NaN,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,2.331040e+09,23310,23310403
43991,2.513200e+09,25132001,2,2,2,1.0,1.0,1.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,2.502030e+09,25020,25020301


In [4]:
# convert hhcode column to string for easy access
df1["hhcode"] = df1["hhcode"].astype("str")

# cleaning important columns
df1["ever_admitted"] = "no"
df1.loc[(df1["s2bq01"] == 1 | 2), "ever_admitted"] = "yes"

df1["currently_enrolled"] = "no"
df1.loc[(df1["s2bq01"] == 3), "currently_enrolled"] = "yes"

In [5]:
# Reducing to the questions of interest:
df1_rel = df1[
    [
        "hhcode",
        "idc",
        "ever_admitted",
        "currently_enrolled",
        "region",
        "province",
        'stratum'
    ]
].copy()

In [6]:
df10 = pd.read_stata("1314_roster.dta")

In [7]:
df10["hhcode"] = df10["hhcode"].astype("str")

df10["sex"] = df10["s1aq04"]
df10["marital_status"] = df10["s1aq07"]

In [8]:
df10_rel = df10[["hhcode", "age", "idc", "sex", "marital_status"]].copy()

In [9]:
years13_14 = pd.merge(df1_rel, df10_rel, on=["hhcode", "idc"], how='inner', indicator=False)

In [10]:
years13_14.loc[(years13_14["region"] == 1), "region"] = "urban"
years13_14.loc[(years13_14["region"] == 2), "region"] = "rural"

# filtering the age for individuals between the ages of 4 - 15
years13_14 = years13_14.loc[(years13_14["age"] > 4) & (years13_14["age"] <= 15)]

years13_14.loc[(years13_14["region"] == 1), "region"] = "urban"
years13_14.loc[(years13_14["region"] == 2), "region"] = "rural"

years13_14["subprovince code"] = years13_14["hhcode"].apply(lambda x: x[0:4])
years13_14["subprovince code"] = years13_14["subprovince code"].astype("int")

#years13_14["province"] = years13_14["province"].astype("int")

years13_14["province"] = years13_14["province"].map(
    {1: "K.P.K", 2: "Punjab", 3: "Sindh", 4: "Balochistan"}
)

# modifying marital status to align with desired output
years13_14["marital_status"].replace(
    [
        "unmarried/never married",
        "currently married",
        "widow",
        " divorced",
        "nikkah has been solemnised but the rukhsati has not taken place code",
    ],
    [1, 2, 3, 4, 5],
    inplace=True,
)

# creating year column with lowest year as value
years13_14["year"] = 2013

years13_14.rename(columns = {'stratum':'code'}, inplace = True)

In [11]:
strat = pd.read_csv('1314_stratum.csv', usecols=['stratum', 'code'])

In [12]:
years13_14 = pd.merge(years13_14, strat, on=['code'], how='inner', indicator=False)

In [13]:
years13_14 = years13_14.drop('code', 1)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [14]:
years13_14.rename(columns = {'stratum':'subprovince'}, inplace = True)

In [15]:
years13_14.sample()

,hhcode,idc,ever_admitted,currently_enrolled,region,province,age,sex,marital_status,subprovince code,year,subprovince
47259,4112100209.0,54,no,no,urban,Balochistan,8,female,1,4112,2013,Chaghai


In [16]:
# PRE-PROCESSING FOR DIFF-IN-DIFF DATA
# convert currently_enrolled from string to integer
years13_14["currently_enrolled"].replace("yes", 1, inplace=True)
years13_14["currently_enrolled"].replace("no", 0, inplace=True)
# check region for anomalies
#years07_08_f["region"].replace(3, "urban", inplace=True)
# aggregate dataset for sample population
df_grp_1 = years13_14.groupby(["sex","subprovince","region"])["currently_enrolled"].count().reset_index()
# aggregate dataset for enrollment total
df_grp_2 = years13_14.groupby(["sex","subprovince","region"])["currently_enrolled"].sum().reset_index()
# merge data set
df_grp_merge = pd.merge(df_grp_1, df_grp_2, on=["sex","subprovince","region"], indicator=True)
# check merge
df_grp_merge._merge.value_counts()

both          472
left_only       0
right_only      0
Name: _merge, dtype: int64

In [17]:
# compute enrollment_rate
df_grp_merge["rate_enrollment"] = df_grp_merge["currently_enrolled_y"]/df_grp_merge["currently_enrolled_x"]
# rename columns
df_grp_merge = df_grp_merge.rename(columns={"currently_enrolled_x":"sample_population", "currently_enrolled_y":"enrolled_total"})
# drop "_merge" column
df_grp_merge.drop(["_merge"], axis=1, inplace=True)
df_grp_merge.sample(5)

,sex,subprovince,region,sample_population,enrolled_total,rate_enrollment
2,male,Attock,rural,112,96,0.857143
202,male,Sibbi,rural,58,40,0.689655
58,male,Harnai,rural,58,40,0.689655
294,female,Harnai,rural,50,33,0.660000
458,female,Tor Garh,rural,105,97,0.923810


In [18]:
# save clean difference-in-difference data set
#df_grp_merge.to_csv("years13_14_merge_diff.csv")

In [19]:
#df_grp_merge.isna().any() # DO THIS BEFORE SUBMITTING THE CSV

In [20]:
years13_14.to_csv("years13_14.csv")